In [5]:
# -*- coding: utf-8 -*-
"""Cifar10Project.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1ia_NH7kj65O1-lC5NipjNMNJMzeTR5FD
"""

# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Function to load CIFAR-10 dataset from binary pickle files
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Define paths to the data files
file_paths = [
    'data_batch_1',
    'data_batch_2',
    'data_batch_3',
    'data_batch_4',
    'data_batch_5'
]

# Detect and set the appropriate device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Load and prepare the training data
batch_data = []
batch_label = []
for file_path in file_paths:
    batch = unpickle(file_path)
    batch_data.append(batch[b'data'])
    batch_label.append(batch[b'labels'])
train_data = np.concatenate(batch_data)
train_labels = np.concatenate(batch_label)
print(train_data.shape, train_labels.shape)

# Load and prepare the test data
test_batch = unpickle('test_batch')
test_batch_data = []
test_batch_labels = []
test_batch_data.append(batch[b'data'])
test_batch_labels.append(batch[b'labels'])
test_data = np.concatenate(test_batch_data)
test_labels = np.concatenate(test_batch_labels)
print("train_data")
print(train_data)
print("test_data")
print(test_data)

# Define the dataset class for CIFAR-10
class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index].reshape(3, 32, 32).transpose(1, 2, 0)
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, label

# Data transformations for training and testing
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Create dataset objects
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=transform_train)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=transform_test)

# Data loaders for handling batches
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# Definition of network architecture components (BasicBlock and Bottleneck)
# Define BasicBlock and Bottleneck here, which are used in the ResNet model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
# Define the ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 32  # Reduced from 64 to adjust to CIFAR-10
        # Additional layers and network initialization
        # Implement the network forward pass

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)

        self.layer3 = self._make_layer(block, 128, max(1, num_blocks[2]-1), stride=2)  # 减少一个block
        self.linear = nn.Linear(512, num_classes)


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
          layers.append(block(self.in_planes, planes, stride))
          self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
def resnet34():
  return ResNet(BasicBlock,[3,4,6,3])
# Instantiate and train the network
model = resnet34().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters is: {total_params}")
if total_params > 5000000:
    print("Warning: Your model has more than 5 million parameters.")

# Training loop

epochs = 50
for epoch in range(epochs):
    print(f"Epoch [{epoch+1}/{epochs}]")
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).long()

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # evaluate model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

# save model
torch.save(model.state_dict(), 'resnet18_cifar10_epoch100.pth')
import csv

# Function to write predictions to a CSV file
def write_predictions_to_csv(test_loader, model, file_name="submission.csv"):
    model.eval()
    ids = []
    labels = []
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(test_loader):
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            ids.extend(range(batch_idx*test_loader.batch_size, (batch_idx+1)*test_loader.batch_size))
            labels.extend(predicted.cpu().numpy())

    # Correct the IDs in case the last batch was smaller
    ids = ids[:len(test_dataset)]

    # Write to CSV file
    with open(file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['ID', 'Labels'])
        for i, label in zip(ids, labels):
            writer.writerow([i, label])

    print(f"Predictions written to {file_name}")

# Generate and save predictions to CSV
write_predictions_to_csv(test_loader, model, "kaggle_submission.csv")



cuda:0
(50000, 3072) (50000,)
train_data
[[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]
 [255 253 253 ...  83  83  84]
 ...
 [ 35  40  42 ...  77  66  50]
 [189 186 185 ... 169 171 171]
 [229 236 234 ... 173 162 161]]
test_data
[[255 252 253 ... 173 231 248]
 [127 126 127 ... 102 108 112]
 [116  64  19 ...   7   6   5]
 ...
 [ 35  40  42 ...  77  66  50]
 [189 186 185 ... 169 171 171]
 [229 236 234 ... 173 162 161]]
Total number of parameters is: 1753258
Epoch [1/50]
Epoch [1/50], Loss: 1.4171
Accuracy of the model on the test images: 54.08%
Epoch [2/50]
Epoch [2/50], Loss: 0.9095
Accuracy of the model on the test images: 68.23%
Epoch [3/50]
Epoch [3/50], Loss: 0.7284
Accuracy of the model on the test images: 76.62%
Epoch [4/50]
Epoch [4/50], Loss: 0.6275
Accuracy of the model on the test images: 79.64%
Epoch [5/50]
Epoch [5/50], Loss: 0.5498
Accuracy of the model on the test images: 81.88%
Epoch [6/50]
Epoch [6/50], Loss: 0.4987
Accuracy of the model on the test images:

In [6]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('kaggle_submission.csv')

# Display the contents of the DataFrame
print(df)


        ID  Labels
0        0       1
1        1       8
2        2       5
3        3       1
4        4       5
...    ...     ...
9995  9995       2
9996  9996       6
9997  9997       9
9998  9998       1
9999  9999       1

[10000 rows x 2 columns]
